In [ ]:
## ODE
import numpy as np
import pandas as pd # for data manipulation
import time
from scipy.integrate import odeint, solve_ivp
from sklearn.metrics import mean_squared_error
import scipy.optimize as optimize

from ipynb.fs.full.myfun_nn import *
from ipynb.fs.defs.myfun_plot import *

# Useful functions

In [ ]:
def time_discretization(t0, tend, deltat=0.05):
    """
    Discretize a time span from t0 to tend with a specified time step.

    Args:
    - t0: Initial time.
    - tend: End time.
    - deltat: Time step (default is 0.05).

    Returns:
    - tspan: Array of discretized time points.
    """

    Nt = round((tend - t0) / deltat) + 1    # Calculate the number of discretization points
    tspan = np.linspace(t0, tend, int(Nt))  # Generate the time span using np.linspace
    
    return tspan

In [ ]:
def seq2scn(df):
    """
    Convert a DataFrame of scenes into a list of scenes.

    Args:
    - df: A pandas DataFrame containing scenes.

    Returns:
    - seq: A list of scenes.
    """
    
    seq = []

    # Iterate over rows to extract input and target for each scene
    for row in df.iterrows():
        scn = row[1]     # Extract the scene from the row
        seq.append(scn)  # Append the scene to the list

    return seq

In [ ]:
def match_timestamps_scene(t, x, deltat=0.05):
    """
    Match the computed solution timestamps to the same timestamps of the scene data.

    Args:
    - t: Computed solution timestamps.
    - x: Computed solution trajectories.
    - deltat: Time step (default is 0.05).

    Returns:
    - t_matched: Matched timestamps.
    - x_matched: Matched trajectories.
    """
    
    # Calculate the factor for matching based on the time step
    factor = int(0.2 / deltat)

    # Match timestamps and trajectories using slicing
    t_matched = np.array(t)[::factor]
    x_matched = np.array([traj[::factor] for traj in x])

    return t_matched, x_matched

In [ ]:
def update_sol_lists(N, tspan_ann, sol_ann, x_list, t_list):
    """
    Update lists containing time and trajectory data after solving an ODE in a sub-interval of a scene.

    Args:
    - N: Number of vehicles.
    - tspan_ann: Timestamps for the sub-interval solution.
    - sol_ann: Computed solution for the sub-interval.
    - x_list: List of vehicle trajectories.
    - t_list: List of timestamps.

    Returns:
    - x_list: Updated list of vehicle trajectories.
    - t_list: Updated list of timestamps.
    """

    # Convert the solution and timestamps to lists
    x_ann = sol_ann.tolist()
    t_ann = tspan_ann[1:]  # Exclude the first recording

    # Add the solution to the correct vehicle trajectory
    for j in range(0, N):
        tmp = x_ann[j][1:]  # Exclude the first recording
        x_list[j] = np.concatenate([x_list[j], tmp])
    
    t_list = np.concatenate([t_list, t_ann]).tolist()

    return x_list, t_list